In [2]:
import tensorflow as tf

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

In [4]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)

In [5]:
print(result)

42


In [6]:
sess.close()

In [7]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [8]:
tf.Session?

In [9]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()

In [10]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [11]:
tf.Variable?

In [12]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

In [13]:
x2.graph is graph

True

In [14]:
x2.graph is tf.get_default_graph

False

In [15]:
graph.as_default?

In [16]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

In [17]:
with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [18]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


In [19]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [20]:
housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

In [48]:
x = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="x")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(x)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, x)), XT), y)

In [22]:
with tf.Session() as sess:
    theta_value = theta.eval()

TensorShape([Dimension(20640), Dimension(9)])

In [23]:
theta_value

array([[-3.7465141e+01],
       [ 4.3573415e-01],
       [ 9.3382923e-03],
       [-1.0662201e-01],
       [ 6.4410698e-01],
       [-4.2513184e-06],
       [-3.7732250e-03],
       [-4.2664889e-01],
       [-4.4051403e-01]], dtype=float32)

In [24]:
from sklearn.preprocessing import StandardScaler

In [25]:
scaler = StandardScaler()

scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [50]:
n_epochs = 1000
learning_rate = 0.01

x = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='x')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(x, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = tf.gradients(mse, [theta])[0]

training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch ", epoch, "MSE = ", mse.eval())
        sess.run(training_op)

    best_theta = theta.eval()

Epoch  0 MSE =  5.284509
Epoch  100 MSE =  0.6638444
Epoch  200 MSE =  0.57079643
Epoch  300 MSE =  0.5591297
Epoch  400 MSE =  0.5517153
Epoch  500 MSE =  0.5460374
Epoch  600 MSE =  0.5416186
Epoch  700 MSE =  0.5381591
Epoch  800 MSE =  0.53543675
Epoch  900 MSE =  0.53328395


In [53]:
theta.shape

TensorShape([Dimension(9), Dimension(1)])

In [54]:
x = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name='x')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name='theta')
print(x.shape)
print(theta.shape)
y_pred = tf.matmul(x, theta, name='predictions')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse, var_list=[theta])

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch ", epoch, "MSE = ", mse.eval())
        sess.run(training_op)

    best_theta = theta.eval()

(20640, 9)
(9, 1)
Epoch  0 MSE =  6.740143
Epoch  100 MSE =  0.8873329
Epoch  200 MSE =  0.74515057
Epoch  300 MSE =  0.6942521
Epoch  400 MSE =  0.65666246
Epoch  500 MSE =  0.62794435
Epoch  600 MSE =  0.6058693
Epoch  700 MSE =  0.58881193
Epoch  800 MSE =  0.5755628
Epoch  900 MSE =  0.5652184


In [29]:
best_theta

array([[ 2.0685523 ],
       [ 0.86888814],
       [ 0.15799043],
       [-0.28009692],
       [ 0.2915235 ],
       [ 0.00926099],
       [-0.04348266],
       [-0.57696277],
       [-0.5493041 ]], dtype=float32)

In [30]:
A = tf.placeholder(tf.float32, shape=(3,))
B = A + 5

with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A : [1, 2, 3]})
    B_val_2 = B.eval(feed_dict={A: [4, 5, 6]})

In [31]:
B_val_1

array([6., 7., 8.], dtype=float32)

In [32]:
B_val_2

array([ 9., 10., 11.], dtype=float32)

In [33]:
x = tf.placeholder(tf.float32, shape=(None, n + 1), name='x')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')

In [34]:
batch_size = 100
n_batches = int(np.ceil(m/batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

In [35]:
from datetime import datetime

now = datetime.utcnow().strftime('%Y%m%d%H%H%S')
root_logdir = 'tf_logs'
logdir = "{}/run-{}/".format(root_logdir, now)

In [36]:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [37]:
best_theta

array([[ 2.0685523 ],
       [ 0.86888814],
       [ 0.15799043],
       [-0.28009692],
       [ 0.2915235 ],
       [ 0.00926099],
       [-0.04348266],
       [-0.57696277],
       [-0.5493041 ]], dtype=float32)

In [38]:
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')

In [39]:
w1 = tf.Variable(tf.random_normal((n_features, 1)), name='weights1')
w2 = tf.Variable(tf.random_normal((n_features, 1)), name='weights2')

b1 = tf.Variable(0.0, name='bias1')
b2 = tf.Variable(0.0, name='bias2')

z1 = tf.add(tf.matmul(X, w1), b1, name='z1')
z2 = tf.add(tf.matmul(X, w2), b1, name='z2')

relu1 = tf.maximum(z1, 0., name='relu1')
relu2 = tf.maximum(z2, 0., name='relu2')

output = tf.add(relu1, relu2, name='output')

In [40]:
def relu(x):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name='weights')
    b = tf.Variable(0.0, name='bias')
    z = tf.add(tf.matmul(X, w), b, name='z')
    return tf.maximum(z, 0, name='relu')

In [41]:
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name='output')

In [42]:
optimizer.minimize?

In [44]:
housing.target

array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894])